# Investment model dispatch patterns
Routines for analyzing dispatch patterns of *pommesinvest* runs

## Package imports

In [ ]:
import warnings
import math
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, 
    aggregate_investment_results, 
    plot_single_dispatch_pattern, 
    plot_generation_and_comsumption_pattern,
    plot_single_investment_variable,
    plot_generation_and_consumption_for_all_cases,
)
from pommesevaluation.global_vars import (
    FUELS, 
    FUELS_RENAMED,
    RES,
    RES_TO_GROUP,
    RES_RENAMED,
    STORAGES,
    STORAGES_RENAMED,
    STORAGES_NEW,
    STORAGES_NEW_RENAMED,
    STORAGES_TO_GROUP,
    DEMAND_RESPONSE,
    DEMAND_RESPONSE_RENAMED,
    ELECTROLYZER,
    ELECTROLYZER_RENAMED,
    LOAD,
    LOAD_RENAMED,
    EVS,
    EVS_RENAMED,
    SHORTAGE_EXCESS,
    SHORTAGE_EXCESS_RENAMED,
    IMPORT_EXPORT,
    IMPORT_EXPORT_RENAMED,
)
from pommesevaluation.tools import update_matplotlib_params

## Parameters and workflow settings

In [ ]:
# General settings
LANGUAGE = "German"  # "German", "English"
start_time_step = "2045-07-06 00:00:00"  # "2037-03-03 16:00:00"
summer_week_start = "-07-10 00:00:00"
winter_week_start = "-01-16 00:00:00"
years_to_evaluate = [2020, 2030, 2045]
time_steps_to_be_considered_in_hours = 168 * 2

PLOT_LABELS = {
    "German": {
        "title": {
            "dispatch": "Dispatch-Situation",
            "combined": "Erzeugungs- und Lastsituation",
        }
    },
    "English": {
        "title": {
            "dispatch": "dispatch situation",
            "combined": "generation and load situation",
        }
    },
}
PLOT_CONFIG = {
    "single": {
        "figsize": (15, 10),
        "figsize_no_legend": (15, 8),
        "bbox": (0.5, -0.27),
    },
    "combined": {
        "figsize": (15, 13),
        "figsize_no_legend": (15, 9),
        "bbox": (0.5, -0.25),
    },
    "peak_load": {
        "figsize": (2, 10),
        "bbox": (0.5, -0.25),
    },
}

# Model configuration
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "50"
sensitivity = ""
dr_scenarios = ["none", "5", "50", "95"]
fuel_price_scenario = "NZE"
emissions_pathway = "long-term"
multi_header = False
countries = [
    "AT",
    "BE",
    "CH",
    "CZ",
    "DE",
    "DK1",
    "DK2",
    "FR",
    "NL",
    "NO1",
    "NO2",
    "NO3",
    "NO4",
    "NO5",
    "PL",
    "SE1",
    "SE2",
    "SE3",
    "SE4",
    "IT",
]
impose_investment_maxima = False

# Paths, filenames and color codes
path_inputs = "./model_inputs/pommesinvest/"
path_results = "./model_results/pommesinvest/"
path_processed_data = "./data_out/"
path_plots = "./plots/"

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    sensitivity_string = ""
    if sensitivity != "":
        warnings.warn(f"CAUTION: Evaluating sensitivity {sensitivity}")
        sensitivity_string = f"_sensitivity_{sensitivity}"
        file_add_on = (
            f"_with_dr_{dr_scenario}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}{sensitivity_string}_production"
        )
    else:
        file_add_on = (
            f"_with_dr_{dr_scenario}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_production"
        )
else:
    sensitivity_string = ""
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_production"
    )
file_extension = ".csv"

filenames_out = {
    "shortages_artificial": {
        dr_scenario: f"{path_processed_data}sources_el_artificial_{dr_scenario}.csv"
        for dr_scenario in dr_scenarios
    },
    "shortages_artificial_ts": {
        dr_scenario: f"{path_processed_data}sources_el_artificial_ts_{dr_scenario}.csv"
        for dr_scenario in dr_scenarios
    }
}

# color codes
FUELS_EXISTING = {
    f: FUELS[f] for f in ["uranium", "lignite", "hardcoal", "mixedfuels", "otherfossil"]
}

FUELS_NEW = {
    f: FUELS[f] for f in ["biomass", "hydrogen", "natgas", "oil", "waste"]
}

ALL_STORAGES = {
    **{s: STORAGES[s] for s in STORAGES},
    **{s: STORAGES_NEW[s] for s in STORAGES_NEW}
}

# filenames for demand response potential information
file_names_demand_response_ts = {
    dr_scen: f"sinks_demand_response_el_ts_{dr_scen}.csv"
    for dr_scen in dr_scenarios if dr_scen != "none"
}

file_names_demand_response_potential_data = {
    dr_scen: [
        f"{dr_cluster}_potential_parameters_{dr_scen}%.csv" 
        for dr_cluster in DEMAND_RESPONSE
    ] for dr_scen in dr_scenarios
}

# Workflow and output configuration
plt.rcParams.update({'font.size': 12})
rounding_precision = 2
amount_of_time_steps = time_steps_to_be_considered_in_hours / int(frequency.split("H")[0])
xtick_frequency = math.floor(amount_of_time_steps / 14)

Configure font sizes for matplotlib

In [ ]:
update_matplotlib_params(
    small_size=14, medium_size=16, large_size=18
)

# Single scenario analyses
Inspect the results for a single scenario model run.
## Read in, preprocess and aggregate data
* Use routine originally developped for investment model to preprocess raw results. Therefore, transpose back and forth.
* Aggregate by fuel. Don't aggregate storages, demand response etc.
* Form distinct data sets:
    * Demand: regular load without demand response baseline consumption and demand response net load
    * Exports and imports: exports from DE to European neighbours, imports vice versa
    * Storages: inflow, outflow and net storage usage derived from these
    * Generators: generation aggregated per fuel
    * Demand Response: upshifts, downshifts and demand response storage level
    * Shortage and excess

In [ ]:
if multi_header:
    header = [0, 1]
else:
    header = 0
production_results_raw = pd.read_csv(
    f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0, header=header
).T
processed_results = preprocess_raw_results(
    production_results_raw, investments=False, multi_header=multi_header
).drop(columns="year").round(rounding_precision)
aggregated_results = aggregate_investment_results(
    processed_results, energy_carriers={**FUELS_EXISTING, **FUELS}, by="energy_carrier", investments=False
).T
del production_results_raw, processed_results

In [ ]:
if dr_scenario != "none":
    # Drop unneeded output for electric vehicles
    to_drop = [col for col in aggregated_results.columns if "bus_ev" in col or "ev_" in col and "outflow" in col]
    aggregated_results.drop(columns=to_drop, inplace=True)

In [ ]:
# Define cols to group
demand_cols = [col for col in aggregated_results.columns if "DE_sink_el" in col and "_excess" not in col]
export_link_cols = [col for col in aggregated_results.columns if "DE_link" in col]
import_link_cols = [col for col in aggregated_results.columns if "link_DE" in col]
power_prices_col = ["DE_bus_el"]

all_demand_response_cols = list(set([
    col for col in aggregated_results.columns for key in DEMAND_RESPONSE if key in col
]))

demand_response_after_cols = [col for col in all_demand_response_cols if "_demand_after" in col]
demand_cols.extend(demand_response_after_cols)

demand_response_other_cols = [
    col for col in all_demand_response_cols 
    if col not in demand_response_after_cols
    # Exclude fictious demand response storage level which can be calculated ex post
    and not "storage_level" in col
]

storages_cols = list(set([
    col for col in aggregated_results.columns 
    for key in STORAGES if key in col
    and "inflow" in col or "outflow" in col
]))
shortage_excess_cols = [
    col for col in aggregated_results.columns if col in ["DE_sink_el_excess", "DE_source_el_shortage"]
]
electrolyzer_cols = [col for col in aggregated_results.columns if "electrolyzer" in col]
foreign_country_cols = list(
    set([
        col for col in aggregated_results.columns for country in countries
        if f"{country}_" in col and country != "DE"
    ])
)
ev_demand_cols = [
    col for col in aggregated_results.columns 
    if col in ["storage_ev_cc_bidirectional_inflow", "storage_ev_cc_unidirectional_inflow", "transformer_ev_uc"]
]
ev_generation_cols = [
    col for col in aggregated_results.columns if col == "transformer_ev_cc_bidirectional_feedback"
]
demand_cols.extend(ev_demand_cols)

generators_cols = [
    col for col in aggregated_results.columns 
    if col not in demand_cols 
    and col not in export_link_cols
    and col not in import_link_cols
    and col not in all_demand_response_cols 
    and col not in storages_cols
    and col not in shortage_excess_cols
    and col not in electrolyzer_cols
    and col not in power_prices_col
    and col not in foreign_country_cols
    and col not in ev_demand_cols
]

# Split overall data set to distinct subsets
demand_pattern = aggregated_results[demand_cols]
export_pattern = aggregated_results[export_link_cols]
import_pattern = aggregated_results[import_link_cols]
net_export_pattern = export_pattern.sum(axis=1) - import_pattern.sum(axis=1)
demand_response_pattern = aggregated_results[demand_response_other_cols]
storages_pattern = aggregated_results[[col for col in storages_cols if not "_ev" in col]]
ev_pattern = aggregated_results[ev_demand_cols + ev_generation_cols]
electrolyzer_pattern = aggregated_results[electrolyzer_cols]
shortage_excess_pattern = aggregated_results[shortage_excess_cols]
generators_pattern = aggregated_results[generators_cols]
generators_pattern["DE_source_biomassEEG"] = generators_pattern["DE_source_biomassEEG"] + generators_pattern["biomass"]
generators_pattern.drop(columns=["biomass"], inplace=True)
try:
    power_prices_pattern = aggregated_results[power_prices_col]
except KeyError:
    pass

# Slightly alter / negate
demand_pattern["DE_sink_el_load"] = demand_pattern["DE_sink_el_load"].clip(0)
storages_pattern.loc[:, [col for col in storages_pattern.columns if "_outflow" in col]] *= (-1)
shortage_excess_pattern.loc[:, "DE_source_el_shortage"] *= (-1)
demand_response_pattern.loc[: , [col for col in demand_response_pattern.columns if "dsm_do" in col]] *= (-1)
ev_pattern.loc[:, [col for col in ev_pattern.columns if "feedback" in col]] *= (-1)

In [ ]:
generators_pattern.sum().sort_values(ascending=False)

## Inspect some long-term results
* Exports and imports patterns
* Demand

### Exports and imports

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
export_pattern.plot(kind="area", stacked=True, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
import_pattern.plot(kind="area", stacked=True, ax=ax)
plt.show()

In [ ]:
if dr_scenario != "none":
    fig, ax = plt.subplots(len(demand_pattern.columns), figsize=(16, 8 * len(demand_pattern.columns)))
    for i, col in enumerate(demand_pattern.columns):
        demand_pattern[col].plot(kind="area", stacked=True, ax=ax[i])
        ax[i].legend(loc="best")
    plt.show()

else:
    fig, ax = plt.subplots(figsize=(16, 8))
    demand_pattern.plot(kind="area", ax=ax)
    plt.show()                           

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
demand_pattern.plot(kind="area", stacked=True, ax=ax)
plt.show()

In [ ]:
demand_pattern.max().sort_values(ascending=False)

## Create simple dispatch plots
### Demand and generation
Create simple area plots

In [ ]:
demand_pattern.describe()

In [ ]:
generators_pattern.describe()

In [ ]:
generators_pattern["mixedfuels"].max()

In [ ]:
colors = {
    **{LOAD_RENAMED[LANGUAGE][l]: LOAD[l] for l in LOAD_RENAMED[LANGUAGE]}, 
    **{
        DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
    },
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
}

to_plot = demand_pattern.rename(
    columns={
        **LOAD_RENAMED[LANGUAGE],
        **{f"{cluster}_demand_after": DEMAND_RESPONSE_RENAMED[LANGUAGE][cluster] for cluster in DEMAND_RESPONSE},
        **EVS_RENAMED[LANGUAGE],
    }
)

plot_single_dispatch_pattern(
    to_plot,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="demand_pattern_dr_scenario",
    title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
    xtick_frequency=xtick_frequency,
    ncol=3,
    language=LANGUAGE,
    dr_scenario=dr_scenario,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
    sensitivity_string=sensitivity_string,
)

In [ ]:
colors = {
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_EXISTING},
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{
        RES_RENAMED[LANGUAGE][r]: RES[r] for r in RES_RENAMED[LANGUAGE]
    },
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
}

to_plot = generators_pattern.copy()
for category, contents in RES_TO_GROUP.items():
    to_plot[category] = to_plot[contents].sum(axis=1)
    to_plot.drop(columns=contents, inplace=True)

to_plot.rename(
    columns={
        **FUELS_RENAMED[LANGUAGE],
        **RES_RENAMED[LANGUAGE],
        **EVS_RENAMED[LANGUAGE],
    },
    inplace=True
)

plot_single_dispatch_pattern(
    to_plot,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="generation_pattern_dr_scenario_dr_scenario",
    title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
    xtick_frequency=xtick_frequency,
    ncol=5,
    language=LANGUAGE,
    dr_scenario=dr_scenario,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
    sensitivity_string=sensitivity_string,
)

### Flexibility options
Area plots, but change in sign

#### Shortage and excess

In [ ]:
shortage_excess_pattern.describe()

In [ ]:
colors = {
    **{SHORTAGE_EXCESS_RENAMED[LANGUAGE][s]: SHORTAGE_EXCESS[s] for s in SHORTAGE_EXCESS_RENAMED[LANGUAGE]}
}

to_plot = shortage_excess_pattern.rename(
    columns={
        **SHORTAGE_EXCESS_RENAMED[LANGUAGE]
    }
)

plot_single_dispatch_pattern(
    to_plot,
    "2020-01-01 00:00:00",
    227758,
    colors,
    save=True,
    path_plots="./plots/",
    filename="shortage_excess_pattern_dr_scenario",
    kind="line",
    title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
    xtick_frequency=math.floor(227758/14),
    ncol=3,
    language=LANGUAGE,
    dr_scenario=dr_scenario,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
    sensitivity_string=sensitivity_string,
)

#### Storages

In [ ]:
storages_pattern.describe()

In [ ]:
storages_pattern

In [ ]:
colors = {
    **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
    **{f"_{STORAGES_RENAMED[LANGUAGE][s]}": STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
}

to_plot = storages_pattern.copy()
for category, contents in STORAGES_TO_GROUP.items():
    to_plot[f"{category}_outflow"] = to_plot[[f"{content}_outflow" for content in contents]].sum(axis=1)
    to_plot[f"{category}_inflow"] = to_plot[[f"{content}_inflow" for content in contents]].sum(axis=1)
    to_plot.drop(columns=[f"{content}_outflow" for content in contents if content != category], inplace=True)
    to_plot.drop(columns=[f"{content}_inflow" for content in contents if content != category], inplace=True)

to_plot.rename(
    columns={
        **{f"{storage}_outflow": f"{STORAGES_RENAMED[LANGUAGE][storage]}_out" for storage in STORAGES},
        **{f"{storage}_outflow": f"{STORAGES_NEW_RENAMED[LANGUAGE][storage]}_out" for storage in STORAGES_NEW},
        **{f"{storage}_inflow": STORAGES_RENAMED[LANGUAGE][storage] for storage in STORAGES},
        **{f"{storage}_inflow": STORAGES_NEW_RENAMED[LANGUAGE][storage] for storage in STORAGES_NEW},
    },
    inplace=True
)
to_plot = to_plot[[col for col in sorted(to_plot.columns)]]
to_plot.rename(
    columns={col: f"_{col.split('_')[0]}" for col in to_plot.columns if col.split("_")[-1] == "out"},
    inplace=True
)

plot_single_dispatch_pattern(
    to_plot,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="storage_pattern_dr_scenario",
    title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
    xtick_frequency=xtick_frequency,
    ncol=3,
    language=LANGUAGE,
    dr_scenario=dr_scenario,
    figsize=PLOT_CONFIG["single"]["figsize"],
    bbox_params=PLOT_CONFIG["single"]["bbox"],
    sensitivity_string=sensitivity_string,
)

#### Demand Response

In [ ]:
if dr_scenario != "none":
    demand_response_pattern.describe()

In [ ]:
if dr_scenario != "none":
    colors = {
        **{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]},
        **{f"_{DEMAND_RESPONSE_RENAMED[LANGUAGE][d]}": DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]},
    }

    to_plot = demand_response_pattern[[col for col in demand_response_pattern.columns if not "dsm_do_shed" in col]].copy()

    to_plot.rename(
        columns={
            **{f"{dr}_dsm_do_shift": f"{DEMAND_RESPONSE_RENAMED[LANGUAGE][dr]}_down" for dr in DEMAND_RESPONSE},
            **{f"{dr}_dsm_up": DEMAND_RESPONSE_RENAMED[LANGUAGE][dr] for dr in DEMAND_RESPONSE},
        },
        inplace=True
    )
    to_plot = to_plot[[col for col in sorted(to_plot.columns)]]
    to_plot.rename(
        columns={col: f"_{col.split('_')[0]}" for col in to_plot.columns if col.split("_")[-1] == "down"},
        inplace=True
    )

    for iter_year in years_to_evaluate:
        if True:  # iter_year != 2045:
            hide_legend_and_xlabel = True
            figsize = PLOT_CONFIG["single"]["figsize_no_legend"]
        else:
            hide_legend_and_xlabel = False
            figsize = PLOT_CONFIG["single"]["figsize"]
        plot_single_dispatch_pattern(
            to_plot,
            f"{iter_year}{winter_week_start}",
            amount_of_time_steps,
            colors,
            save=True,
            path_plots="./plots/",
            filename="demand_response_pattern_dr_scenario",
            title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
            xtick_frequency=int(xtick_frequency/2),
            ncol=3,
            language=LANGUAGE,
            dr_scenario=dr_scenario,
            figsize=figsize,
            bbox_params=PLOT_CONFIG["single"]["bbox"],
            hide_legend_and_xlabel=hide_legend_and_xlabel,
            sensitivity_string=sensitivity_string,
        )

In [ ]:
if dr_scenario != "none":
    for iter_year in years_to_evaluate:
        if iter_year != 2045:
            hide_legend_and_xlabel = True
            figsize = PLOT_CONFIG["single"]["figsize_no_legend"]
        else:
            hide_legend_and_xlabel = False
            figsize = PLOT_CONFIG["single"]["figsize"]
        plot_single_dispatch_pattern(
            to_plot,
            f"{iter_year}{summer_week_start}",
            amount_of_time_steps,
            colors,
            save=True,
            path_plots="./plots/",
            filename="demand_response_pattern_dr_scenario",
            title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
            xtick_frequency=int(xtick_frequency/2),
            ncol=3,
            language=LANGUAGE,
            dr_scenario=dr_scenario,
            figsize=PLOT_CONFIG["single"]["figsize"],
            bbox_params=PLOT_CONFIG["single"]["bbox"],
            hide_legend_and_xlabel=hide_legend_and_xlabel,
            sensitivity_string=sensitivity_string,
        )

In [ ]:
if dr_scenario != "none":
    demand_response_pattern.loc[
        "2037-03-08 00:00":"2037-03-09 23:00", 
        ["ind_cluster_shift_only_dsm_do_shift", "ind_cluster_shift_only_dsm_up"]
    ]

#### Elevtric Vehicles (EVs)

In [ ]:
if dr_scenario != "none":
    colors = {
        **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
    }

    to_plot = ev_pattern.rename(
        columns={
            **EVS_RENAMED[LANGUAGE],
        }
    )

    plot_single_dispatch_pattern(
        to_plot,
        start_time_step,
        amount_of_time_steps,
        colors,
        save=True,
        path_plots="./plots/",
        filename="ev_pattern_dr_scenario",
        title=PLOT_LABELS[LANGUAGE]["title"]["dispatch"],
        xtick_frequency=xtick_frequency,
        ncol=4,
        language=LANGUAGE,
        dr_scenario=dr_scenario,
        figsize=PLOT_CONFIG["single"]["figsize"],
        bbox_params=PLOT_CONFIG["single"]["bbox"],
        sensitivity_string=sensitivity_string,
    )

## Generation mix vs. load
Plot generation and load in a joint graph which stacks generation sources in the positive and demand sources in the negative range

In [ ]:
generation_and_load = pd.concat(
    [
        -pd.DataFrame(data=net_export_pattern, columns=["import/export"]), 
        generators_pattern, 
        -demand_pattern, 
        -storages_pattern,
        -shortage_excess_pattern, 
        -electrolyzer_pattern
    ], 
    axis=1,
)
generation_and_load = generation_and_load.loc[:,~generation_and_load.columns.duplicated()].copy()
to_plot = generation_and_load.copy()
for category, contents in RES_TO_GROUP.items():
    to_plot[category] = to_plot[contents].sum(axis=1)
    to_plot.drop(columns=contents, inplace=True)

PHS_cols = [col for col in to_plot.columns if "PHS" in col]
battery_cols = [col for col in to_plot.columns if "battery" in col]
to_plot["PHS"] = to_plot[PHS_cols].sum(axis=1)
to_plot["battery"] = to_plot[battery_cols].sum(axis=1)

to_plot.drop(columns=PHS_cols + battery_cols, inplace=True)

In [ ]:
colors = {
    **{IMPORT_EXPORT_RENAMED[LANGUAGE][i]: IMPORT_EXPORT[i] for i in IMPORT_EXPORT_RENAMED[LANGUAGE]},
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_EXISTING}, 
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{RES_RENAMED[LANGUAGE][r]: RES[r] for r in RES_RENAMED[LANGUAGE]},
    **{LOAD_RENAMED[LANGUAGE][l]: LOAD[l] for l in LOAD_RENAMED[LANGUAGE]},
    **{
        DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
    },
    **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
    **{SHORTAGE_EXCESS_RENAMED[LANGUAGE][s]: SHORTAGE_EXCESS[s] for s in SHORTAGE_EXCESS_RENAMED[LANGUAGE]},
    **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
}

to_plot.rename(
    columns={
        **IMPORT_EXPORT_RENAMED[LANGUAGE],
        **FUELS_RENAMED[LANGUAGE],
        **RES_RENAMED[LANGUAGE],
        **LOAD_RENAMED[LANGUAGE],
        **{f"{cluster}_demand_after": DEMAND_RESPONSE_RENAMED[LANGUAGE][cluster] for cluster in DEMAND_RESPONSE},
        **STORAGES_RENAMED[LANGUAGE],
        **EVS_RENAMED[LANGUAGE],
        **SHORTAGE_EXCESS_RENAMED[LANGUAGE],
        **{f"{electrolyzer}_new_built": ELECTROLYZER_RENAMED[LANGUAGE][electrolyzer] for electrolyzer in ELECTROLYZER},
    },
    inplace=True
)

for iter_year in years_to_evaluate:
    if True:  # iter_year != 2045:  # if False (show legend every time)
        hide_legend_and_xlabel = True
        figsize = PLOT_CONFIG["combined"]["figsize_no_legend"]
    else:
        hide_legend_and_xlabel = False
        figsize = PLOT_CONFIG["combined"]["figsize"]
    plot_generation_and_comsumption_pattern(
        to_plot,
        f"{iter_year}{winter_week_start}",
        amount_of_time_steps,
        colors,
        filename="combined_pattern_dr_scenario",
        title=PLOT_LABELS[LANGUAGE]["title"]["combined"],
        language=LANGUAGE,
        dr_scenario=dr_scenario,
        figsize=figsize,
        bbox_params=PLOT_CONFIG["combined"]["bbox"],
        hide_legend_and_xlabel=hide_legend_and_xlabel,
        ncol=3,
        sensitivity_string=sensitivity_string,
    )

In [ ]:
for iter_year in years_to_evaluate:
    if iter_year != 2045:  # if False (show legend every time)
        hide_legend_and_xlabel = True
        figsize = PLOT_CONFIG["combined"]["figsize_no_legend"]
    else:
        hide_legend_and_xlabel = False
        figsize = PLOT_CONFIG["combined"]["figsize"]
    plot_generation_and_comsumption_pattern(
        to_plot,
        f"{iter_year}{summer_week_start}",
        amount_of_time_steps,
        colors,
        filename="combined_pattern_dr_scenario",
        title=PLOT_LABELS[LANGUAGE]["title"]["combined"],
        language=LANGUAGE,
        dr_scenario=dr_scenario,
        figsize=figsize,
        bbox_params=PLOT_CONFIG["combined"]["bbox"],
        hide_legend_and_xlabel=hide_legend_and_xlabel,
        ncol=3,
        sensitivity_string=sensitivity_string,
    )

In [ ]:
gen_cols = [col for col in generation_and_load.columns if (generation_and_load[col] > -1E3).all() and (generation_and_load[col].sum() > 0)]
dem_cols = [col for col in generation_and_load.columns if (generation_and_load[col] < 1E3).all() and (generation_and_load[col].sum() < 0)]

In [ ]:
# TODO: Find and fix error in postprocessing leading to remaining imbalance (due to the model ensuring electricity bus to be balanced, this definitely has to be a postprocessing bug!)
generation_and_load.sum(axis=1).plot()

## Analyze peak load coverage
* Analyze coverage of residual load for situation with highest residual load as well as situation with minimum RES infeed over two weeks
* Obtain planned demand from input data

In [ ]:
res_generation = generators_pattern[["DE_source_ROR", "DE_source_solarPV", "DE_source_windoffshore", "DE_source_windonshore"]].sum(axis=1)
classical_demand = demand_pattern[["DE_sink_el_load"]]
ev_demand = ev_pattern[ev_demand_cols]

In [ ]:
if dr_scenario != "none":
    dr_baseline_ts = pd.read_csv(f"{path_inputs}{file_names_demand_response_ts[dr_scenario]}", index_col=0)
    max_cap_ts = pd.DataFrame()
    for file in file_names_demand_response_potential_data[dr_scenario]:
        cluster_name = file.rsplit("_", 3)[0]
        max_cap_ts[cluster_name] = pd.read_csv(f"{path_inputs}{file}", index_col=0)["max_cap"]

    to_concat = []
    for iter_year in range(2020, 2046):
        to_concat.append(
            dr_baseline_ts.loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59"
            ].mul(max_cap_ts.loc[iter_year])
        )
    dr_baseline_demand = pd.concat(to_concat)
else:
    dr_baseline_demand = pd.DataFrame(index=classical_demand.index)

In [ ]:
# Combine demand and calculate residual load
all_demands = pd.concat([classical_demand, dr_baseline_demand, ev_demand, electrolyzer_pattern], axis=1)
total_demand = all_demands.sum(axis=1)
residual_load = total_demand - res_generation

In [ ]:
# Extract maximum residual load indices and values
max_residual_load = pd.DataFrame(columns=["value"])
for iter_year in residual_load.index.str[:4].unique():
    max_residual = residual_load.loc[residual_load.index.str[:4] == iter_year].max()
    index_location = pd.Index(residual_load).get_loc(max_residual)
    index_value = residual_load.to_frame().iloc[index_location].name
    max_residual_load.loc[index_value, "value"] = max_residual

In [ ]:
# Extract situation with minimum RES infeed for the course of two weeks
duration = 168

min_res_infeed = pd.DataFrame(columns=["value"])
for iter_year in res_generation.index.str[:4].unique():
    gen_rolling = res_generation.loc[res_generation.index.str[:4] == iter_year].rolling(duration).sum()
    min_infeed = gen_rolling.min()
    end_index_location = pd.Index(gen_rolling).get_loc(min_infeed)
    start_index_value = gen_rolling.to_frame().iloc[end_index_location - duration].name
    min_res_infeed.loc[start_index_value, "value"] = min_infeed

In [ ]:
# Analyze residual peak load coverage
colors = {
    **{IMPORT_EXPORT_RENAMED[LANGUAGE][i]: IMPORT_EXPORT[i] for i in IMPORT_EXPORT_RENAMED[LANGUAGE]},
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_EXISTING}, 
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{RES_RENAMED[LANGUAGE][r]: RES[r] for r in RES_RENAMED[LANGUAGE]},
    **{LOAD_RENAMED[LANGUAGE][l]: LOAD[l] for l in LOAD_RENAMED[LANGUAGE]},
    **{
        DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
    },
    **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
    **{SHORTAGE_EXCESS_RENAMED[LANGUAGE][s]: SHORTAGE_EXCESS[s] for s in SHORTAGE_EXCESS_RENAMED[LANGUAGE]},
    **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
}

to_plot = generation_and_load.copy()
for category, contents in RES_TO_GROUP.items():
    to_plot[category] = to_plot[contents].sum(axis=1)
    to_plot.drop(columns=contents, inplace=True)

PHS_cols = [col for col in to_plot.columns if "PHS" in col]
battery_cols = [col for col in to_plot.columns if "battery" in col]
to_plot["PHS"] = to_plot[PHS_cols].sum(axis=1)
to_plot["battery"] = to_plot[battery_cols].sum(axis=1)

to_plot.drop(columns=PHS_cols + battery_cols, inplace=True)
to_plot.rename(
    columns={
        **IMPORT_EXPORT_RENAMED[LANGUAGE],
        **FUELS_RENAMED[LANGUAGE],
        **RES_RENAMED[LANGUAGE],
        **LOAD_RENAMED[LANGUAGE],
        **{f"{cluster}_demand_after": DEMAND_RESPONSE_RENAMED[LANGUAGE][cluster] for cluster in DEMAND_RESPONSE},
        **STORAGES_RENAMED[LANGUAGE],
        **EVS_RENAMED[LANGUAGE],
        **SHORTAGE_EXCESS_RENAMED[LANGUAGE],
        **{f"{electrolyzer}_new_built": ELECTROLYZER_RENAMED[LANGUAGE][electrolyzer] for electrolyzer in ELECTROLYZER},
    },
    inplace=True
)

for idx in max_residual_load.index:
    # Scale as a dirty fix
    total_generation = to_plot.loc[idx].clip(lower=0).sum()
    total_load = -to_plot.loc[idx].clip(upper=0).sum()
    dem_cols = to_plot.loc[idx, to_plot.loc[idx] < 0].index
    to_plot.loc[[idx], dem_cols] = to_plot.loc[[idx], dem_cols] * total_generation / total_load
    
    plot_generation_and_comsumption_pattern(
        to_plot,
        idx,
        0,
        colors,
        title=PLOT_LABELS[LANGUAGE]["title"]["combined"],
        filename="residual_peak_load_single_hour_dr_scenario",
        kind="bar",
        single_hour=True,
        language=LANGUAGE,
        dr_scenario=dr_scenario,
        figsize=PLOT_CONFIG["peak_load"]["figsize"],
        bbox_params=PLOT_CONFIG["peak_load"]["bbox"],
        sensitivity_string=sensitivity_string,
    )
    print(to_plot.loc[idx].sum())
    print(total_generation / total_load)

In [ ]:
# Analyze load coverage over the course of minimum RES infeed period
colors = {
    **{IMPORT_EXPORT_RENAMED[LANGUAGE][i]: IMPORT_EXPORT[i] for i in IMPORT_EXPORT_RENAMED[LANGUAGE]},
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_EXISTING}, 
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{RES_RENAMED[LANGUAGE][r]: RES[r] for r in RES_RENAMED[LANGUAGE]},
    **{LOAD_RENAMED[LANGUAGE][l]: LOAD[l] for l in LOAD_RENAMED[LANGUAGE]},
    **{
        DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
    },
    **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
    **{SHORTAGE_EXCESS_RENAMED[LANGUAGE][s]: SHORTAGE_EXCESS[s] for s in SHORTAGE_EXCESS_RENAMED[LANGUAGE]},
    **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
}

to_plot = generation_and_load.copy()
for category, contents in RES_TO_GROUP.items():
    to_plot[category] = to_plot[contents].sum(axis=1)
    to_plot.drop(columns=contents, inplace=True)

PHS_cols = [col for col in to_plot.columns if "PHS" in col]
battery_cols = [col for col in to_plot.columns if "battery" in col]
to_plot["PHS"] = to_plot[PHS_cols].sum(axis=1)
to_plot["battery"] = to_plot[battery_cols].sum(axis=1)

to_plot.drop(columns=PHS_cols + battery_cols, inplace=True)
to_plot.rename(
    columns={
        **IMPORT_EXPORT_RENAMED[LANGUAGE],
        **FUELS_RENAMED[LANGUAGE],
        **RES_RENAMED[LANGUAGE],
        **LOAD_RENAMED[LANGUAGE],
        **{f"{cluster}_demand_after": DEMAND_RESPONSE_RENAMED[LANGUAGE][cluster] for cluster in DEMAND_RESPONSE},
        **STORAGES_RENAMED[LANGUAGE],
        **EVS_RENAMED[LANGUAGE],
        **SHORTAGE_EXCESS_RENAMED[LANGUAGE],
        **{f"{electrolyzer}_new_built": ELECTROLYZER_RENAMED[LANGUAGE][electrolyzer] for electrolyzer in ELECTROLYZER},
    },
    inplace=True
)

for idx in min_res_infeed.index:
    plot_generation_and_comsumption_pattern(
        to_plot,
        idx,
        duration,
        colors,
        title=PLOT_LABELS[LANGUAGE]["title"]["combined"],
        filename="low_res_infeed_dr_scenario",
        figsize=PLOT_CONFIG["combined"]["figsize"],
        language=LANGUAGE,
        bbox_params=PLOT_CONFIG["combined"]["bbox"],
        dr_scenario=dr_scenario,
        sensitivity_string=sensitivity_string,
    )

## Analyze annual generation per energy carrier
Calculate / show annual sums for generation per energy carrier

In [ ]:
colors = {
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_EXISTING},
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{
        RES_RENAMED[LANGUAGE][r]: RES[r] for r in RES_RENAMED[LANGUAGE]
    },
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
}

to_plot = generators_pattern.copy()
for category, contents in RES_TO_GROUP.items():
    to_plot[category] = to_plot[contents].sum(axis=1)
    to_plot.drop(columns=contents, inplace=True)

to_plot.rename(
    columns={
        **FUELS_RENAMED[LANGUAGE],
        **RES_RENAMED[LANGUAGE],
        **EVS_RENAMED[LANGUAGE],
    },
    inplace=True
)

all_years = [iter_year for iter_year in to_plot.index.str[:4].unique()]
annual_generation = pd.DataFrame(index=all_years, columns=to_plot.columns)
for iter_year in all_years:
    to_plot_iter_year = to_plot.loc[
        to_plot.index.str[:4] == iter_year
    ].sum()
    annual_generation.loc[iter_year] = to_plot_iter_year / 1000
    
# Hack: Use stacked plot function from investment results inspection
plot_single_investment_variable(
    annual_generation.T,
    "generation",
    colors=colors,
    group=False,
    aggregation="energy carrier",
    save=True,
    filename="results_annual_generation_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 800000],
    language=LANGUAGE,
    exclude_unit=True,
    sensitivity_string=sensitivity_string,
)

In [ ]:
im_ex = pd.DataFrame(columns=["value"])
for iter_year in all_years:
    im_ex.loc[iter_year] = generation_and_load["import/export"].loc[
        generation_and_load["import/export"].index.str[:4] == iter_year
    ].sum() / 1000

In [ ]:
im_ex.plot()

# Multi scenario analyses
## Extract dispatch results

In [ ]:
aggregated_results = {}

for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        file_add_on = (
            f"_with_dr_{dr_scenario}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_production"
        )
    else:
        file_add_on = (
            f"_no_dr_50_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_production"
        )
    if multi_header:
        header = [0, 1]
    else:
        header = 0
    
    production_results_raw = pd.read_csv(
        f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0, header=header,
    ).T
    processed_results = preprocess_raw_results(
        production_results_raw, investments=False, multi_header=multi_header
    ).drop(columns="year").round(rounding_precision)
    aggregated_results[dr_scenario] = aggregate_investment_results(
        processed_results, 
        energy_carriers={**FUELS_EXISTING, **FUELS}, 
        by="energy_carrier", 
        investments=False
    ).T
    if dr_scenario != "none":
        # Drop unneeded output for electric vehicles
        to_drop = [col for col in aggregated_results[dr_scenario].columns if "bus_ev" in col or "ev_" in col and "outflow" in col]
        aggregated_results[dr_scenario].drop(columns=to_drop, inplace=True)
    
del production_results_raw, processed_results

In [ ]:
demand_pattern = dict()
export_pattern = dict()
import_pattern = dict()
net_export_pattern = dict()
demand_response_pattern = dict()
storages_pattern = dict()
ev_pattern = dict()
electrolyzer_pattern = dict()
shortage_excess_pattern = dict()
generators_pattern = dict()
power_prices_pattern = dict()

# Define cols to group
for dr_scenario in dr_scenarios:
    demand_cols = [col for col in aggregated_results[dr_scenario].columns if "DE_sink_el" in col and "_excess" not in col]
    export_link_cols = [col for col in aggregated_results[dr_scenario].columns if "DE_link" in col]
    import_link_cols = [col for col in aggregated_results[dr_scenario].columns if "link_DE" in col]
    power_prices_col = ["DE_bus_el"]

    all_demand_response_cols = list(set([
        col for col in aggregated_results[dr_scenario].columns for key in DEMAND_RESPONSE if key in col
    ]))

    demand_response_after_cols = [col for col in all_demand_response_cols if "_demand_after" in col]
    demand_cols.extend(demand_response_after_cols)

    demand_response_other_cols = [
        col for col in all_demand_response_cols 
        if col not in demand_response_after_cols
        # Exclude fictious demand response storage level which can be calculated ex post
        and not "storage_level" in col
    ]

    storages_cols = list(set([
        col for col in aggregated_results[dr_scenario].columns 
        for key in STORAGES if key in col
        and "inflow" in col or "outflow" in col
    ]))
    shortage_excess_cols = [
        col for col in aggregated_results[dr_scenario].columns if col in ["DE_sink_el_excess", "DE_source_el_shortage"]
    ]
    electrolyzer_cols = [col for col in aggregated_results[dr_scenario].columns if "electrolyzer" in col]
    foreign_country_cols = list(
        set([
            col for col in aggregated_results[dr_scenario].columns for country in countries
            if f"{country}_" in col and country != "DE"
        ])
    )
    ev_demand_cols = [
        col for col in aggregated_results[dr_scenario].columns 
        if col in ["storage_ev_cc_bidirectional_inflow", "storage_ev_cc_unidirectional_inflow", "transformer_ev_uc"]
    ]
    ev_generation_cols = [
        col for col in aggregated_results[dr_scenario].columns if col == "transformer_ev_cc_bidirectional_feedback"
    ]
    demand_cols.extend(ev_demand_cols)

    generators_cols = [
        col for col in aggregated_results[dr_scenario].columns 
        if col not in demand_cols 
        and col not in export_link_cols
        and col not in import_link_cols
        and col not in all_demand_response_cols 
        and col not in storages_cols
        and col not in shortage_excess_cols
        and col not in electrolyzer_cols
        and col not in power_prices_col
        and col not in foreign_country_cols
        and col not in ev_demand_cols
    ]

    # Split overall data set to distinct subsets
    demand_pattern[dr_scenario] = aggregated_results[dr_scenario][demand_cols]
    export_pattern[dr_scenario] = aggregated_results[dr_scenario][export_link_cols]
    import_pattern[dr_scenario] = aggregated_results[dr_scenario][import_link_cols]
    net_export_pattern[dr_scenario] = export_pattern[dr_scenario].sum(axis=1) - import_pattern[dr_scenario].sum(axis=1)
    demand_response_pattern[dr_scenario] = aggregated_results[dr_scenario][demand_response_other_cols]
    storages_pattern[dr_scenario] = aggregated_results[dr_scenario][[col for col in storages_cols if col not in ev_demand_cols]]
    ev_pattern[dr_scenario] = aggregated_results[dr_scenario][ev_demand_cols + ev_generation_cols]
    electrolyzer_pattern[dr_scenario] = aggregated_results[dr_scenario][electrolyzer_cols]
    shortage_excess_pattern[dr_scenario] = aggregated_results[dr_scenario][shortage_excess_cols]
    generators_pattern[dr_scenario] = aggregated_results[dr_scenario][generators_cols]
    generators_pattern[dr_scenario]["DE_source_biomassEEG"] = generators_pattern[dr_scenario]["DE_source_biomassEEG"] + generators_pattern[dr_scenario]["biomass"]
    generators_pattern[dr_scenario].drop(columns=["biomass"], inplace=True)
    try:
        power_prices_pattern[dr_scenario] = aggregated_results[dr_scenario][power_prices_col]
    except KeyError:
        pass

    # Slightly alter / negate
    demand_pattern[dr_scenario]["DE_sink_el_load"] = demand_pattern[dr_scenario]["DE_sink_el_load"].clip(0)
    storages_pattern[dr_scenario].loc[:, [col for col in storages_pattern[dr_scenario].columns if "_outflow" in col]] *= (-1)
    shortage_excess_pattern[dr_scenario].loc[:, "DE_source_el_shortage"] *= (-1)
    demand_response_pattern[dr_scenario].loc[: , [col for col in demand_response_pattern[dr_scenario].columns if "dsm_do" in col]] *= (-1)
    ev_pattern[dr_scenario].loc[:, [col for col in ev_pattern[dr_scenario].columns if "feedback" in col]] *= (-1)

## Analyze peak load coverage

In [ ]:
max_residual_load_dict = dict()
start_time_steps = dict()

for dr_scenario in dr_scenarios:
    res_generation = generators_pattern[dr_scenario][["DE_source_ROR", "DE_source_solarPV", "DE_source_windoffshore", "DE_source_windonshore"]].sum(axis=1)
    classical_demand = demand_pattern[dr_scenario][["DE_sink_el_load"]]
    if dr_scenario != "none":
        cols_to_use = ev_demand_cols
    else:
        cols_to_use = []
    ev_demand = ev_pattern[dr_scenario][cols_to_use]

    if dr_scenario != "none":
        dr_baseline_ts = pd.read_csv(f"{path_inputs}{file_names_demand_response_ts[dr_scenario]}", index_col=0)
        max_cap_ts = pd.DataFrame()
        for file in file_names_demand_response_potential_data[dr_scenario]:
            cluster_name = file.rsplit("_", 3)[0]
            max_cap_ts[cluster_name] = pd.read_csv(f"{path_inputs}{file}", index_col=0)["max_cap"]

        to_concat = []
        for iter_year in range(2020, 2046):
            to_concat.append(
                dr_baseline_ts.loc[
                    f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59"
                ].mul(max_cap_ts.loc[iter_year])
            )
        dr_baseline_demand = pd.concat(to_concat)
    else:
        dr_baseline_demand = pd.DataFrame(index=classical_demand.index)

    # Combine demand and calculate residual load
    all_demands = pd.concat([classical_demand, dr_baseline_demand, ev_demand, electrolyzer_pattern[dr_scenario]], axis=1)
    total_demand = all_demands.sum(axis=1)
    residual_load = total_demand - res_generation

    # Extract maximum residual load indices and values
    max_residual_load = pd.DataFrame(columns=["value"])
    for iter_year in residual_load.index.str[:4].unique():
        max_residual = residual_load.loc[residual_load.index.str[:4] == iter_year].max()
        index_location = pd.Index(residual_load).get_loc(max_residual)
        index_value = residual_load.to_frame().iloc[index_location].name
        max_residual_load.loc[index_value, "value"] = max_residual
    
    max_residual_load_dict[dr_scenario] = max_residual_load
    start_time_steps[dr_scenario] = index_value

In [ ]:
to_plot = dict()
for dr_scenario in dr_scenarios:
    generation_and_load = pd.concat(
        [
            -pd.DataFrame(data=net_export_pattern[dr_scenario], columns=["import/export"]), 
            generators_pattern[dr_scenario], 
            -demand_pattern[dr_scenario], 
            -storages_pattern[dr_scenario], 
            -ev_pattern[dr_scenario], 
            -shortage_excess_pattern[dr_scenario], 
            -electrolyzer_pattern[dr_scenario]
        ], 
        axis=1,
    )
    generation_and_load = generation_and_load.loc[:,~generation_and_load.columns.duplicated()].copy()
    to_plot[dr_scenario] = generation_and_load.copy()
    for category, contents in RES_TO_GROUP.items():
        to_plot[dr_scenario][category] = to_plot[dr_scenario][contents].sum(axis=1)
        to_plot[dr_scenario].drop(columns=contents, inplace=True)

    PHS_cols = [col for col in to_plot[dr_scenario].columns if "PHS" in col]
    battery_cols = [col for col in to_plot[dr_scenario].columns if "battery" in col]
    to_plot[dr_scenario]["PHS"] = to_plot[dr_scenario][PHS_cols].sum(axis=1)
    to_plot[dr_scenario]["battery"] = to_plot[dr_scenario][battery_cols].sum(axis=1)

    to_plot[dr_scenario].drop(columns=PHS_cols + battery_cols, inplace=True)

In [ ]:
# Analyze residual peak load coverage
colors = {
    **{IMPORT_EXPORT_RENAMED[LANGUAGE][i]: IMPORT_EXPORT[i] for i in IMPORT_EXPORT_RENAMED[LANGUAGE]},
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_EXISTING}, 
    **{FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS_NEW},
    **{RES_RENAMED[LANGUAGE][r]: RES[r] for r in RES_RENAMED[LANGUAGE]},
    **{LOAD_RENAMED[LANGUAGE][l]: LOAD[l] for l in LOAD_RENAMED[LANGUAGE]},
    **{
        DEMAND_RESPONSE_RENAMED[LANGUAGE][d]: DEMAND_RESPONSE[d] for d in DEMAND_RESPONSE_RENAMED[LANGUAGE]
    },
    **{STORAGES_RENAMED[LANGUAGE][s]: STORAGES[s] for s in STORAGES_RENAMED[LANGUAGE]},
    **{EVS_RENAMED[LANGUAGE][e]: EVS[e] for e in EVS_RENAMED[LANGUAGE]},
    **{SHORTAGE_EXCESS_RENAMED[LANGUAGE][s]: SHORTAGE_EXCESS[s] for s in SHORTAGE_EXCESS_RENAMED[LANGUAGE]},
    **{ELECTROLYZER_RENAMED[LANGUAGE][e]: ELECTROLYZER[e] for e in ELECTROLYZER_RENAMED[LANGUAGE]},
}

for dr_scenario in dr_scenarios:
    to_plot[dr_scenario].rename(
        columns={
            **IMPORT_EXPORT_RENAMED[LANGUAGE],
            **FUELS_RENAMED[LANGUAGE],
            **RES_RENAMED[LANGUAGE],
            **LOAD_RENAMED[LANGUAGE],
            **{f"{cluster}_demand_after": DEMAND_RESPONSE_RENAMED[LANGUAGE][cluster] for cluster in DEMAND_RESPONSE},
            **STORAGES_RENAMED[LANGUAGE],
            **EVS_RENAMED[LANGUAGE],
            **SHORTAGE_EXCESS_RENAMED[LANGUAGE],
            **{f"{electrolyzer}_new_built": ELECTROLYZER_RENAMED[LANGUAGE][electrolyzer] for electrolyzer in ELECTROLYZER},
        },
        inplace=True
    )

plot_generation_and_consumption_for_all_cases(
    to_plot,
    start_time_steps,
    0,
    colors,
    fig_height=10,
    subplot_width=4,
    save=True,
    path_plots="./plots/",
    filename="residual_peak_load",
    place_legend_below=True,
    ncol=4,
    bbox_params=(-1.8, -0.6),
    wspace=0.5,
    y_label_pos=(0.04, 0.5),
    language="German",
    hide_legend_and_xlabel=False,
    scale=True,
    sharey=True,
)

## Shortage events for other countries
* Inspect occurences of shortage for countries other than Germany.
* Prepare as a model input:
    * Artificial shortage sources with a fixed profile are added to pommesdata to prevent shortages.
    * Realized shortage profiles are extracted and used as fixed profile for pommesdata.

Extraction and inspection:
* Extract shortage patterns
* Visualize shortage time series

In [ ]:
shortages = {}
fig, axs = plt.subplots(len(dr_scenarios), 1, figsize=(15, 5 * len(dr_scenarios)))

for no, dr_scenario in enumerate(dr_scenarios):
    shortage_cols = [
        col for col in aggregated_results[dr_scenario].columns if "shortage" in col
        and "artificial" not in col
    ]
    shortages[dr_scenario] = aggregated_results[dr_scenario][shortage_cols]
    _ = shortages[dr_scenario].plot(ax=axs[no])
    _ = axs[no].set_title(f"shortage events for scenario {dr_scenario}")
    
plt.show()

In [ ]:
shortages[dr_scenario].max()

## Maximum shortage capacities
* Extract maximum storage capacities
* add oemof.solph-relevant pieces of information and save to file

In [ ]:
for dr_scenario in dr_scenarios:
    shortages_max = pd.DataFrame(index=shortages[dr_scenario].columns)
    shortages_max.index.name = "label"
    shortages_max["to"] = shortages_max.index.str.split("_", expand=True).get_level_values(0) + "_bus_el"
    shortages_max["maximum"] = shortages[dr_scenario].max()
    shortages_max.index = shortages_max.index + "_artificial"
    shortages_max = shortages_max.loc[shortages_max.maximum != 0]
    shortages_max.to_csv(filenames_out["shortages_artificial"][dr_scenario])

## Shortage profiles
* Extract normalized shortage profiles
* Store time series to file

In [ ]:
for dr_scenario in dr_scenarios:
    shortages_profile = shortages[dr_scenario].div(shortages[dr_scenario].max())
    shortages_profile.columns = [col + "_artificial" for col in shortages_profile.columns]
    shortages_profile = shortages_profile[[
        col for col in shortages_profile.columns
        if shortages_profile[col].notna().all()
    ]]
    shortages_profile.to_csv(filenames_out["shortages_artificial_ts"][dr_scenario])